 # 模仿前面的写一个UT

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import jieba
import codecs
import collections
import sys
from operator import itemgetter
import tqdm

import time
import numpy as np
import matplotlib.pyplot as plt

# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [2]:
DIR_PATH="/home/xujingzhou/en-zh"


 ### 分词、清洗、建立SubwordTextEncoder
 > 建立过程在subwordTextEncoder_*.py中完成了

 模型读取与展示

In [3]:
tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.load_from_file(DIR_PATH+'/en')
tokenizer_zh = tfds.deprecated.text.SubwordTextEncoder.load_from_file(DIR_PATH+'/zh')
sample_string = 'How a hot day!'
tokenized_string=tokenizer_en.encode(sample_string)
print(tokenized_string)
original_string=tokenizer_en.decode(tokenized_string)
print(original_string)
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer_en.decode([ts])))
# print("-----------")
# sample_string='你好,世界!'
# tokenized_string=tokenizer_zh.encode(sample_string)
# print(tokenized_string)
# original_string=tokenizer_zh.decode(tokenized_string)
# print(original_string)
# for ts in tokenized_string:
#   print ('{} ----> {}'.format(ts, tokenizer_zh.decode([ts])))
#YesYesYes!!


[32346, 15384, 7, 12149, 2540, 32307]
How a hot day!
32346 ----> H
15384 ----> ow 
7 ----> a 
12149 ----> hot 
2540 ----> day
32307 ----> !


 可以看到这个模型还是有一定的问题的，比如中文标点的表示方法等等，后续可能需要通过改进分词器来提高模型效果。

In [4]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64


 将开始和结束标记（token）添加到输入和目标。

In [11]:
def encode(lang1, lang2):
  lang1 = [tokenizer_en.vocab_size] + tokenizer_en.encode(
      lang1.numpy()) + [tokenizer_en.vocab_size+1]

  lang2 = [tokenizer_zh.vocab_size] + tokenizer_zh.encode(
      lang2.numpy()) + [tokenizer_zh.vocab_size+1]
  
  return lang1, lang2


 这一步似乎并不需要

In [5]:
MAX_LENGTH = 80


In [6]:
def filter_max_length(x, y, max_length=MAX_LENGTH):
  return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)


 创建encode函数

In [7]:
def tf_encode(en, zh):
  result_en, result_zh = tf.py_function(encode, [en, zh], [tf.int64, tf.int64])
  result_en.set_shape([None])
  result_zh.set_shape([None])

  return result_en, result_zh


In [8]:
#这里需要创建train——examples和test_examples：需要用到dataset
with codecs.open(DIR_PATH+'/train.zh','r','utf-8') as f:
    zh_data=f.readlines()
    #讲中文标点转为英文标点
    zh_data=[i.replace('。', '.').replace('，', ',').replace('？', '?').replace('！', '!').replace('、',',') for i in zh_data]
    zh_data=tf.data.Dataset.from_tensor_slices(zh_data)
with codecs.open(DIR_PATH+'/train.en','r','utf-8') as f:
    en_data=[]
    # #转小写
    for line in f:
      en_data.append(line.lower())
    # en_data=f.readlines()
    en_data=tf.data.Dataset.from_tensor_slices(en_data)
train_data=tf.data.Dataset.zip((en_data,zh_data))


In [9]:
#val_Data
with codecs.open(DIR_PATH+'/test.zh','r','utf-8') as f:
    zh_data=f.readlines()
    zh_data=[i.replace('。', '.').replace('，', ',').replace('？', '?').replace('！', '!').replace('、',',') for i in zh_data]
    zh_data=tf.data.Dataset.from_tensor_slices(zh_data)
with codecs.open(DIR_PATH+'/test.en','r','utf-8') as f:
    en_data=[]
    for line in f:
      en_data.append(line.lower())
    # en_data=f.readlines()
    en_data=tf.data.Dataset.from_tensor_slices(en_data)
val_data=tf.data.Dataset.zip((en_data,zh_data))
print("loaded")

loaded


In [12]:
train_dataset = train_data.map(tf_encode)#英中
train_dataset = train_dataset.filter(filter_max_length)
# 将数据集缓存到内存中以加快读取速度。
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = val_data.map(tf_encode)
val_dataset = val_dataset.filter(filter_max_length).padded_batch(BATCH_SIZE)


In [92]:
pt_batch, en_batch = next(iter(train_dataset))
pt_batch, en_batch


(<tf.Tensor: shape=(64, 65), dtype=int64, numpy=
 array([[32530,    22,  2320, ...,     0,     0,     0],
        [32530,  6098, 32320, ...,     0,     0,     0],
        [32530,     1,   346, ...,     0,     0,     0],
        ...,
        [32530,   126, 12364, ...,     0,     0,     0],
        [32530,     9,   595, ...,     0,     0,     0],
        [32530,   147,  8193, ...,     0,     0,     0]])>,
 <tf.Tensor: shape=(64, 76), dtype=int64, numpy=
 array([[31162,    42,   224, ...,     0,     0,     0],
        [31162,  1247, 30952, ...,     0,     0,     0],
        [31162,   605,    62, ...,     0,     0,     0],
        ...,
        [31162,   281,  1058, ...,     0,     0,     0],
        [31162,   371,   392, ...,     0,     0,     0],
        [31162,  5446, 12959, ...,     0,     0,     0]])>)

In [13]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates


In [14]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  
  # 将 sin 应用于数组中的偶数索引（indices）；2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  
  # 将 cos 应用于数组中的奇数索引；2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
  pos_encoding = angle_rads[np.newaxis, ...]
    
  return tf.cast(pos_encoding, dtype=tf.float32)


In [15]:
pos_encoding = positional_encoding(50, 512)
print (pos_encoding.shape)

# plt.pcolormesh(pos_encoding[0], cmap='RdBu')
# plt.xlabel('Depth')
# plt.xlim((0, 512))
# plt.ylabel('Position')
# plt.colorbar()
# plt.show()


(1, 50, 512)


In [16]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # 添加额外的维度来将填充加到
  # 注意力对数（logits）。
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)


In [17]:
# x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
# create_padding_mask(x)


In [18]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)


In [19]:
# x = tf.random.uniform((1, 3))
# temp = create_look_ahead_mask(x.shape[1])
# temp


In [20]:
def scaled_dot_product_attention(q, k, v, mask):
  """计算注意力权重。
  q, k, v 必须具有匹配的前置维度。
  k, v 必须有匹配的倒数第二个维度，例如：seq_len_k = seq_len_v。
  虽然 mask 根据其类型（填充或前瞻）有不同的形状，
  但是 mask 必须能进行广播转换以便求和。
  
  参数:
    q: 请求的形状 == (..., seq_len_q, depth)
    k: 主键的形状 == (..., seq_len_k, depth)
    v: 数值的形状 == (..., seq_len_v, depth_v)
    mask: Float 张量，其形状能转换成
          (..., seq_len_q, seq_len_k)。默认为None。
    
  返回值:
    输出，注意力权重
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # 缩放 matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # 将 mask 加入到缩放的张量上。
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax 在最后一个轴（seq_len_k）上归一化，因此分数
  # 相加等于1。
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights


In [21]:
def print_out(q, k, v):
  temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
  print ('Attention weights are:')
  print (temp_attn)
  print ('Output is:')
  print (temp_out)


In [22]:
np.set_printoptions(suppress=True)

temp_k = tf.constant([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[   1,0],
                      [  10,0],
                      [ 100,5],
                      [1000,6]], dtype=tf.float32)  # (4, 2)

# 这条 `请求（query）符合第二个`主键（key）`，
# 因此返回了第二个`数值（value）`。
# temp_q = tf.constant([[0, 10, 0]], dtype=tf.float32)  # (1, 3)
# print_out(temp_q, temp_k, temp_v)

# # %%
# # 这条请求符合重复出现的主键（第三第四个），
# # 因此，对所有的相关数值取了平均。
# temp_q = tf.constant([[0, 0, 10]], dtype=tf.float32)  # (1, 3)
# print_out(temp_q, temp_k, temp_v)


In [23]:
# 这条请求符合第一和第二条主键，
# 因此，对它们的数值去了平均。
# temp_q = tf.constant([[10, 10, 0]], dtype=tf.float32)  # (1, 3)
# print_out(temp_q, temp_k, temp_v)

# # %%
# temp_q = tf.constant([[0, 0, 10], [0, 10, 0], [10, 10, 0]], dtype=tf.float32)  # (3, 3)
# print_out(temp_q, temp_k, temp_v)


In [24]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """分拆最后一个维度到 (num_heads, depth).
    转置结果使得形状为 (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights


In [25]:
# temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
# y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
# out, attn = temp_mha(y, k=y, q=y, mask=None)
# out.shape, attn.shape


In [109]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

class ACT(tf.keras.Model):
    
    def __init__(self, batch_size, length, hidden_size):
        super(ACT, self).__init__()
        
        self.halting_probability = tf.zeros((batch_size, length), name='halting_probability')
        self.remainders = tf.zeros((batch_size, length), name="remainder")
        self.n_updates = tf.zeros((batch_size, length), name="n_updates")
        
    def call(self, pondering, halt_threshold):
        still_running = tf.cast(tf.less(self.halting_probability, 1.0), tf.float32)

        # mask
        new_halted = tf.greater(self.halting_probability + pondering * still_running, halt_threshold)
        new_halted = tf.cast(new_halted, tf.float32) * still_running

        # update mask
        still_running_now = tf.less_equal(self.halting_probability + pondering * still_running, halt_threshold)
        still_running_now = tf.cast(still_running_now, tf.float32) * still_running

        # update halting probability
        self.halting_probability += pondering * still_running

        # update remainders and halting probability
        self.remainders += new_halted * (1 - self.halting_probability)
        self.halting_probability += new_halted * self.remainders

        # update times
        self.n_updates += still_running + new_halted

        # calc update weights
        update_weights = pondering * still_running + new_halted * self.remainders
        update_weights = tf.expand_dims(update_weights, -1)
        
        return update_weights
    
    def should_continue(self, threshold) -> bool:
        result=tf.reduce_any(tf.less(self.halting_probability, threshold))
        # result = tf.cast(result, tf.bool)
        #展示result的数据类型
        return result

def act_layer(pondering, halt_threshold,halting_probability,remainders,n_updates):
    still_running = tf.cast(tf.less(halting_probability, 1.0), tf.float32)
    # mask
    new_halted = tf.greater(halting_probability + pondering * still_running, halt_threshold)
    new_halted = tf.cast(new_halted, tf.float32) * still_running

    # update mask
    still_running_now = tf.less_equal(halting_probability + pondering * still_running, halt_threshold)
    still_running_now = tf.cast(still_running_now, tf.float32) * still_running

    # update halting probability
    halting_probability += pondering * still_running

    # update remainders and halting probability
    remainders += new_halted * (1 - halting_probability)
    halting_probability += new_halted * remainders

    # update times
    n_updates += still_running + new_halted

    # calc update weights
    update_weights = pondering * still_running + new_halted * remainders
    update_weights = tf.expand_dims(update_weights, -1)
    # print(update_weights.shape)
    
    return update_weights,halting_probability,remainders,n_updates

def should_continue(threshold,halting_probability) -> bool:
    result=tf.reduce_any(tf.less(halting_probability, threshold))
    result = tf.cast(result, tf.bool)
    return result


In [117]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1,max_act_steps=20,act_epsilon=0.01):#这么大？
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    self.max_act_steps = max_act_steps
    self.act_epsilon=act_epsilon
    
    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)
    self.pondering_layer = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid, use_bias=True, bias_initializer=tf.constant_initializer(1.0))
    
    self.self_attention_layers = MultiHeadAttention(d_model, num_heads)
    self.ffn_layers = point_wise_feed_forward_network(d_model, dff)

    self.layer_norm1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layer_norm2=tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
  
    self.output_norm=tf.keras.layers.LayerNormalization(epsilon=1e-6)
      
  def call(self, x, training, mask):
    # print("x.shape:",x.shape)
    def f1(state,halting_probability,remainders,n_updates):
      # print("x")
      return state,halting_probability,remainders,n_updates

    def f2(state,previous_state,training,seq_len,step,mask,halting_probability,remainders,n_updates):
      # print("y")
      state+=self.pos_encoding[:,:seq_len,:]
      signal=self.pos_encoding[0,:self.max_act_steps,:]
      signal=tf.expand_dims(signal[step,:],axis=0)
      # print(signal.shape)
      # print(state.shape)
      state+=signal
      # print("x")
      pondering=self.pondering_layer(state)
      pondering=tf.squeeze(pondering,axis=-1)
      # print("x")
      update_weight,halting_probability,remainders,n_updates=act_layer(pondering,1-self.act_epsilon,halting_probability,remainders,n_updates)
      # print("x")
      state1=self.layer_norm1(state)
      state1,_=self.self_attention_layers(state1,state1,state1,mask)#这个mask有用吗？
      state=self.dropout1(state1+state,training=training)
      # print("x")
      state2=self.layer_norm2(state)
      state2=self.ffn_layers(state2)
      state=self.dropout2(state2+state,training=training)
      # print("x")
      state=state*update_weight+previous_state*(1-update_weight)
      return state,halting_probability,remainders,n_updates

    seq_len = tf.shape(x)[1]
    batch_size= tf.shape(x)[0]

    halting_probability = tf.zeros((batch_size, seq_len), name='halting_probability')
    remainders = tf.zeros((batch_size, seq_len), name="remainder")
    n_updates = tf.zeros((batch_size, seq_len), name="n_updates")
    
    # 将嵌入和位置编码相加。
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    state=x
    previous_state=tf.zeros_like(x) 
    # print("0")
    for step in range(self.max_act_steps):
      # print(state.shape)
      state,halting_probability,remainders,n_updates=tf.cond(should_continue(1-self.act_epsilon,halting_probability),
        lambda:f2(state,previous_state,training,seq_len,step,mask,halting_probability,remainders,n_updates),
        lambda:f1(state,halting_probability,remainders,n_updates))
      # print("2")
      previous_state=state      
    return self.output_norm(state),n_updates,remainders



In [28]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1,max_act_steps=20,act_epsilon=0.01):#这么大行吗？
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    self.max_act_steps = max_act_steps
    self.act_epsilon=act_epsilon
    
    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)
    self.pondering_layer = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid, use_bias=True, bias_initializer=tf.constant_initializer(1.0))
    
    self.self_attention_layers = MultiHeadAttention(d_model, num_heads)
    self.ffn_layers = point_wise_feed_forward_network(d_model, dff)

    self.layer_norm1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layer_norm2=tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
  
    self.output_norm=tf.keras.layers.LayerNormalization(epsilon=1e-6)

        
  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]
    batch_size= tf.shape(x)[0]
    act=ACT(batch_size,seq_len,self.d_model)
    # 将嵌入和位置编码相加。
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    state=x
    previous_state=tf.zeros_like(x) 
    # print("0")
    for step in range(self.max_act_steps):
      if act.should_continue(1-self.act_epsilon):
        break
      state+=self.pos_encoding[:,:seq_len,:]
      #时间步
      signal=self.pos_encoding[0,:self.max_act_steps,:]
      signal=tf.expand_dims(signal[step,:],axis=0)
      # print(signal.shape)
      # print(state.shape)
      state+=signal

      pondering=self.pondering_layer(state)
      pondering=tf.squeeze(pondering,axis=-1)

      update_weight=act(pondering,1-self.act_epsilon)
      #多头自注意力
      state1=self.layer_norm1(state)
      state1,_=self.self_attention_layers(state1,state1,state1,mask)#这个mask有用吗？
      state=self.dropout1(state1+state,training=training)

      state2=self.layer_norm2(state)
      state2=self.ffn_layers(state2)
      state=self.dropout2(state2+state,training=training)

      state=state*update_weight+previous_state*(1-update_weight)
      previous_state=state
      
    return self.output_norm(state),act.n_updates,act.remainders

In [29]:
sample_encoder = Encoder(num_layers=2, d_model=512, num_heads=8, 
                         dff=2048, input_vocab_size=8500,
                         maximum_position_encoding=10000)

sample_encoder_output,_,_ = sample_encoder(tf.random.uniform((64, 62)), 
                                       training=False, mask=None)

print (sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)


(64, 62, 512)


In [135]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1,max_act_steps=20,act_epsilon=0.01):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    self.max_act_steps = max_act_steps
    self.act_epsilon=act_epsilon
    
    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
    self.pondering_layer = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid, use_bias=True, bias_initializer=tf.constant_initializer(1.0))
    
    self.self_attention_layers = MultiHeadAttention(d_model, num_heads)
    self.enc_dec_attention_layers = MultiHeadAttention(d_model, num_heads)
    self.ffn_layers = point_wise_feed_forward_network(d_model, dff)

    self.layer_norm1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layer_norm2=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layer_norm3=tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)
    self.dropout=tf.keras.layers.Dropout(rate)
  
    self.output_norm=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
    def f1(state,halting_probability,remainders,n_updates):
      # print("x")
      return state,halting_probability,remainders,n_updates
    def f2(state,previous_state,training,seq_len,step,halting_probability,remainders,n_updates,enc_output,look_ahead_mask,padding_mask):
      state+=self.pos_encoding[:,:seq_len,:]
      # print("x")
      signal=self.pos_encoding[0,:self.max_act_steps,:]
      signal=tf.expand_dims(signal[step,:],axis=0)
      state+=signal
      # print("x")
      pondering=self.pondering_layer(state)
      pondering=tf.squeeze(pondering,axis=-1)
      # print("x")
      update_weight,halting_probability,remainders,n_updates=act_layer(pondering,1-self.act_epsilon,halting_probability,remainders,n_updates)
      # print("x")
      state1=self.layer_norm1(state)
      state1,attention_weights=self.self_attention_layers(state1,state1,state1,look_ahead_mask)
      state=self.dropout1(state1+state,training=training)
      # print("x")
      state2=self.layer_norm2(state)
      state2,attention_weights=self.enc_dec_attention_layers(enc_output,enc_output,state2,padding_mask)
      state=self.dropout2(state2+state,training=training)
      # print("x")
      state3=self.layer_norm3(state)
      state3=self.ffn_layers(state3)
      state=self.dropout3(state3+state,training=training)
      # print("x")
      state=state*update_weight+previous_state*(1-update_weight)
      return state,halting_probability,remainders,n_updates

    seq_len = tf.shape(x)[1]
    batch_size= tf.shape(x)[0]

    halting_probability = tf.zeros((batch_size, seq_len), name='halting_probability')
    remainders = tf.zeros((batch_size, seq_len), name="remainder")
    n_updates = tf.zeros((batch_size, seq_len), name="n_updates")

    x=self.embedding(x)
    x*=tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x=self.dropout(x,training=training)
    
    state=x
    previous_state=tf.zeros_like(x)
    # print(state.shape)
    for step in range(self.max_act_steps):
      # print(step)
      state,halting_probability,remainders,n_updates=tf.cond(should_continue(1-self.act_epsilon,halting_probability),
        lambda:f2(state,previous_state,training,seq_len,step,halting_probability,remainders,n_updates,enc_output,look_ahead_mask,padding_mask),
        lambda:f1(state,halting_probability,remainders,n_updates))
      previous_state=state

    state=self.output_norm(state)
    return state,n_updates,remainders


In [31]:
# sample_decoder = Decoder(num_layers=2, d_model=512, num_heads=8, 
#                          dff=2048, target_vocab_size=8000,
#                          maximum_position_encoding=5000)

# output, attn = sample_decoder(tf.random.uniform((64, 26)), 
#                               enc_output=sample_encoder_output, 
#                               training=False, look_ahead_mask=None, 
#                               padding_mask=None)

# output.shape, attn['decoder_layer2_block2'].shape


In [136]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               target_vocab_size, pe_input, pe_target, rate=0.1,act_loss_weight=0.01,max_act_steps=20,act_epsilon=0.01):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                           input_vocab_size, pe_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                           target_vocab_size, pe_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    self.act_loss_weight=act_loss_weight
    
  def call(self, inp, tar, training, enc_padding_mask, 
           look_ahead_mask, dec_padding_mask):
    # print("encoder")
    # print(inp.shape)
    enc_output, enc_ponders, enc_remainders = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)
    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    print("decoder")
    dec_output, dec_ponders, dec_remainders= self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)
    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)
    enc_act_loss = tf.reduce_mean(enc_ponders + enc_remainders)
    dec_act_loss = tf.reduce_mean(dec_ponders + dec_remainders)
    act_loss = enc_act_loss + dec_act_loss
    return final_output,act_loss


In [137]:
sample_transformer = Transformer(
    num_layers=2, d_model=512, num_heads=8, dff=2048, 
    input_vocab_size=8500, target_vocab_size=8000, 
    pe_input=10000, pe_target=6000)

temp_input = tf.random.uniform((64, 62))
temp_target = tf.random.uniform((64, 26))
with tf.GradientTape() as tape:
    fn_out, _ = sample_transformer(temp_input, temp_target, training=True, 
                                enc_padding_mask=None, 
                                look_ahead_mask=None,
                                dec_padding_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)


decoder
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


TensorShape([64, 26, 8000])

In [34]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8

input_vocab_size = tokenizer_en.vocab_size + 2
target_vocab_size = tokenizer_zh.vocab_size + 2
dropout_rate = 0.1


In [35]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
    
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


In [36]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)


In [37]:
# temp_learning_rate_schedule = CustomSchedule(d_model)

# plt.plot(temp_learning_rate_schedule(tf.range(40000, dtype=tf.float32)))
# plt.ylabel("Learning Rate")
# plt.xlabel("Train Step")


In [38]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


In [39]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_mean(loss_)


In [40]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')


In [41]:
def create_masks(inp, tar):
  # 编码器填充遮挡
  enc_padding_mask = create_padding_mask(inp)
  
  # 在解码器的第二个注意力模块使用。
  # 该填充遮挡用于遮挡编码器的输出。
  dec_padding_mask = create_padding_mask(inp)
  
  # 在解码器的第一个注意力模块使用。
  # 用于填充（pad）和遮挡（mask）解码器获取到的输入的后续标记（future tokens）。
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
  return enc_padding_mask, combined_mask, dec_padding_mask


In [138]:
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          input_vocab_size, target_vocab_size, 
                          pe_input=input_vocab_size, 
                          pe_target=target_vocab_size,
                          rate=dropout_rate)


In [139]:
checkpoint_path = "/home/xujingzhou/checkpoint_u1"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# 如果检查点存在，则恢复最新的检查点。
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')


In [140]:
EPOCHS = 1


In [141]:
# 该 @tf.function 将追踪-编译 train_step 到 TF 图中，以便更快地
# 执行。该函数专用于参数张量的精确形状。为了避免由于可变序列长度或可变
# 批次大小（最后一批次较小）导致的再追踪，使用 input_signature 指定
# 更多的通用形状。

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]

@tf.function()
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
  with tf.GradientTape() as tape:
    predictions,act_loss= transformer(inp, tar_inp, 
                                 True, 
                                 enc_padding_mask, 
                                 combined_mask, 
                                 dec_padding_mask)
    loss = loss_function(tar_real, predictions)+act_loss
  print("loss")
  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  train_loss(loss)
  train_accuracy(tar_real, predictions)


In [142]:
for epoch in range(EPOCHS):
  start = time.time()
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  
  # inp -> portuguese, tar -> english
  for (batch, (inp, tar)) in enumerate(val_dataset):
    train_step(inp, tar)
    
    if batch % 50 == 0:
      print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
    break
      
  if (epoch + 1) % 2 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    
  print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

  print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))


decoder
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
decoder
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Epoch 1 Batch 0 Loss 10.4931 Accuracy 0.0000
Epoch 1 Loss 10.4931 Accuracy 0.0000
Time taken for 1 epoch: 116.052738904953 secs



In [ ]:
#en和zh弄反了呜呜呜呜呜呜呜呜呜呜呜！！
def evaluate(inp_sentence):
  start_token = [tokenizer_en.vocab_size]
  end_token = [tokenizer_en.vocab_size + 1]
  
  # 输入语句是葡萄牙语，增加开始和结束标记
  inp_sentence = start_token + tokenizer_en.encode(inp_sentence) + end_token
  encoder_input = tf.expand_dims(inp_sentence, 0)
  
  # 因为目标是英语，输入 transformer 的第一个词应该是
  # 英语的开始标记。
  decoder_input = [tokenizer_zh.vocab_size]
  output = tf.expand_dims(decoder_input, 0)
    
  for i in range(MAX_LENGTH):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)
  
    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions,_= transformer(encoder_input, 
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)
    
    # 从 seq_len 维度选择最后一个词
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    
    # 如果 predicted_id 等于结束标记，就返回结果
    if predicted_id == tokenizer_zh.vocab_size+1:
      return tf.squeeze(output, axis=0)
    
    # 连接 predicted_id 与输出，作为解码器的输入传递到解码器。
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)


In [ ]:
#这个是错误的错误！是妥协！
# def evaluate(inp_sentence):
#   start_token = [tokenizer_zh.vocab_size]
#   end_token = [tokenizer_zh.vocab_size + 1]
  
#   # 输入语句是葡萄牙语，增加开始和结束标记
#   inp_sentence = start_token + tokenizer_en.encode(inp_sentence) + end_token
#   encoder_input = tf.expand_dims(inp_sentence, 0)
  
#   # 因为目标是英语，输入 transformer 的第一个词应该是
#   # 英语的开始标记。
#   decoder_input = [tokenizer_en.vocab_size]
#   output = tf.expand_dims(decoder_input, 0)
    
#   for i in range(MAX_LENGTH):
#     enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
#         encoder_input, output)
  
#     # predictions.shape == (batch_size, seq_len, vocab_size)
#     predictions, attention_weights = transformer(encoder_input, 
#                                                  output,
#                                                  False,
#                                                  enc_padding_mask,
#                                                  combined_mask,
#                                                  dec_padding_mask)
    
#     # 从 seq_len 维度选择最后一个词
#     predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

#     predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    
#     # 如果 predicted_id 等于结束标记，就返回结果
#     if predicted_id == tokenizer_en.vocab_size+1:
#       return tf.squeeze(output, axis=0), attention_weights
    
#     # 连接 predicted_id 与输出，作为解码器的输入传递到解码器。
#     output = tf.concat([output, predicted_id], axis=-1)

#   return tf.squeeze(output, axis=0), attention_weights


In [ ]:
def translate(sentence):
  result= evaluate(sentence)
  print(result)
  predicted_sentence = tokenizer_zh.decode([i for i in result 
                                            if i < tokenizer_zh.vocab_size])  

  print('Input: {}'.format(sentence))
  print('Predicted translation: {}'.format(predicted_sentence))

